In [11]:
%run full_scraper.ipynb

Done page 193


KeyboardInterrupt: 

KeyboardInterrupt: 

In [2]:
lots_dict = {}
categories = ['Distillery', 'Age', 'Vintage', 'Region', 'Bottler', 'Cask Type', 'Bottled Strength', 'Bottle Size',
              'Distillery Status']
ua = UserAgent()
date_format1 = "%m.%d.%y"
date_format2 = "%d.%m.%y"
per_page = 40

In [12]:
search = input("What do you want to search for?")
search_txt = search.replace(" ", '+')

search_url = f"https://whiskyauctioneer.com/auction-search?text={search_txt}&sort=field_reference_field_end_date+DESC"

What do you want to search for? pappy van winkle


In [13]:
def search_scraper():
    '''Scrapes data from Whiskey Auctioneer and saves to dictionary'''
    
    # more_new_data = True

    # URL of the first page of results
    url_stem = search_url
    
    try:
        live_url = "https://whiskyauctioneer.com/current-auction"
        page = requests.get(live_url, headers={'user-agent': ua.random})
        soup = BeautifulSoup(page.content, "html.parser")
        start_page = math.floor(int(re.findall(r'\d+', soup.find("p", class_ = "left").text)[0]) / per_page)-1
        i = start_page
        
    except:
        i = 1
    
    # Loop through remaining pages until you get to where there aren't any more
    while validators.url(live_url)==True:
        
        if i == 0:
            url = url_stem
        else:
            url = url_stem + f'?page={i}'

        page = requests.get(url, headers={'user-agent': ua.random})
        soup = BeautifulSoup(page.content, "html.parser")
        
        
        result_1 = soup.find("div", class_ ="views-row views-row-1 views-row-odd views-row-first producthomepage")
        result = result_1
        
        # Now we need to iterate through the lots on the page
        while result != None:
            
            lot_dict = {}
            
            lot_num = int(re.findall(r'\d+', result.find("span", class_ = "lotnumber label-lot").text)[0])
            if lot_num in list(scraped_data_cur.index):
                
                more_new_data = False
                result = result.nextSibling
                continue
            
            # Skip this iteration if the lot is part of a current auction:
            if "Bid Now" in result.text:
                result = result.nextSibling
                continue
                 
            lot_dict["name"] = result.find("a")["_title"]
            lot_dict["price"] = int(''.join(re.findall(r'\d+', result.find("div", class_ = "lotwin cru").text)))
            if "Reserve not met" in result.find("div", class_ = "lotwin cru").text:
                lot_dict['reserve_met'] = 0
            elif "Reserve has been met" in result.find("div", class_ = "lotwin cru").text:
                lot_dict['reserve_met'] = 1
            try:
                lot_dict['end_date'] = datetime.strptime(result.find("div", 
                                    class_ = "enddatein").text.split(':')[1].strip(), date_format1).date()
            except:
                lot_dict['end_date'] = datetime.strptime(result.find("div", 
                                    class_ = "enddatein").text.split(':')[1].strip(), date_format2).date()
            lot_num = int(re.findall(r'\d+', result.find("span", class_ = "lotnumber label-lot").text)[0])
               
            # Get lot site
            lot_url = result.find("a", href=True)['href']
            
            lot_page = requests.get(lot_url, headers={'user-agent': ua.random})
            lot_soup = BeautifulSoup(lot_page.content, "html.parser")
            
            
            try:
                child = lot_soup.find("div", class_ = "whiskyproduct").findChild()
            except:
                print(lot_url)
                result = result.nextSibling
                continue
            # Iterate through children to get all info
            while child:
                
                for val in categories:
                    if val in child.text:
                        cat = val
                        
                value = str(child.text).replace("\xa0", '').split(':')[1]
                
                lot_dict[cat] = value
                
                child = child.nextSibling
                
            prod_details = lot_soup.find("div", {"class": "field-item even", 
                                                 "property": "content:encoded"}).text.replace("\xa0", '')
            
            lot_dict['details'] = prod_details
                
            lots_dict[lot_num] = lot_dict
            
            # This iterates the loop to the next lot
            result = result.nextSibling
            
            time.sleep(3)
        
        print(f"Done page {i}")
        i += 1
        time.sleep(3)
         
        
    return lots_dict
        

In [14]:
search_scraper()

Done page 191
Done page 192
Done page 193
Done page 194
Done page 195


KeyboardInterrupt: 

In [18]:
scraped_data_cur = pd.DataFrame.from_dict(lots_dict, orient = "index")

In [20]:
scraped_data_cur.to_csv("scraped_data_cur.csv")